In [ ]:
# OpenCV - https://opencv.org/
# MediaPipe - https://google.github.io/mediapipe/solutions/hands.html
# numPy - https://numpy.org/
# Random - https://docs.python.org/3/library/random.html
import cv2
import mediapipe as mp
import numpy as np
import uuid
import time
import os
from random import randrange

#Set size of camera frame
####
wCam, hCam = 1280, 720
####


##############################################################
# Credits
#
# Hand Landmark list creation from https://youtu.be/9iEPzbG-xLE
# Author:  Murtaza Hassan
# Title: Hand Tracking 30 FPS using CPU | OpenCV Python (2021) | Computer Vision
#
# Alternative MediaPipe setup https://youtu.be/vQZ4IvB07ec
# Author: Nicholas Renotte
# Title: AI Hand Pose Estimation with MediaPipe and Python
##############################################################

class handDetector():
    # Initialize MediaPipe Hand object
    def __init__(self, mode=False,
                 maxHands = 2,
                 detectionCon=0.8,
                 trackCon=0.6):

        # Create object which has its own variable
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        # Necessary objects for initialization
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode,self.maxHands,self.detectionCon,self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils

    def findHands(self, img, draw=False):

        # Convert image to RGB from BGR becuase this class can only use RGB
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Process image
        self.results = self.hands.process(imgRGB)

        # print(results.multi_hand_landmarks)

        # This is the original functionality which renders the hand landmarks
        # We are not using the below code because we want to be able to determine left and right sepearately
        # But not deleting code because it is elpful reference
        # if self.results.multi_hand_landmarks:
        #     for handLms in self.results.multi_hand_landmarks:
        #         if draw:
        #             self.mpDraw.draw_landmarks(img, handLms, self.mpHands.HAND_CONNECTIONS)
        return img

    def getRightHand(self, img):

        # Uses multi_handedness to determine if right hand

        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)

        right = None

        # link landmark with handedness - https://stackoverflow.com/questions/67455791/mediapipe-python-link-landmark-with-handedness/67928368#67928368
        if self.results.multi_handedness:
            # Check if landmark is detected
            # print('Handedness:', self.results.multi_handedness['label'])
            for idx, handLms in enumerate(self.results.multi_hand_landmarks):
                ...
                # print(self.results.multi_handedness[idx].classification[0].label)
                if self.results.multi_handedness[idx].classification[0].label == "Right":
                    right = self.results.multi_handedness[idx].classification[0].label

        return right

    def getLeftHand(self, img):

        # Uses multi_handedness to determine if left hand

        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)

        left = None

        if self.results.multi_handedness:
            # print('Handedness:', self.results.multi_handedness['label'])
            for idx, handLms in enumerate(self.results.multi_hand_landmarks):
                ...
                # print(self.results.multi_handedness[idx].classification[0].label)
                if self.results.multi_handedness[idx].classification[0].label == "Left":
                    left = self.results.multi_handedness[idx].classification[0].label

        return left

    def findPositionLeft(self, img, handNo=0, draw=False):

        # Finds positions of left hand only

        # List will contain all landmark postitions for left hand
        lmListLeft = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]

            if self.results.multi_handedness:
                # print('Handedness:', self.results.multi_handedness['label'])
                for id, lm in enumerate(myHand.landmark):
                    # Get the height, width, and channels of the image
                    h, w, c = img.shape
                    # Find the position of the center
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    # Print the id number and x/y position
                    # print(id, cx, cy)
                    lmListLeft.append([id, cx, cy])
                    # if draw:
                    #     cv2.circle(img, (cx, cy), 5, (0, 0, 0), cv2.FILLED)

        return lmListLeft

    def findPositionRight(self, img, handNo=0, draw=False):

        # Finds positions of right hand only

        # List will contain all landmark postitions for left hand
        lmListRight = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]

            if self.results.multi_handedness:
                # print('Handedness:', self.results.multi_handedness['label'])
                for id, lm in enumerate(myHand.landmark):

                    h, w, c = img.shape

                    cx, cy = int(lm.x * w), int(lm.y * h)

                    lmListRight.append([id, cx, cy])
                    # if id == 0:
                    # if draw:
                    #     cv2.circle(img, (cx, cy), 5, (255, 255, 255), cv2.FILLED)

        return lmListRight

    def render(self, img, r, g, b, r_, g_, b_, circle_size, line_size):

        # Renders connections and circles which changes depending on state

        if self.results.multi_hand_landmarks:

            for num, hand in enumerate(self.results.multi_hand_landmarks):
                # Get landmark information
                # More info on mediapipe landmarks: https://google.github.io/mediapipe/solutions/hands.html
                self.mpDraw.draw_landmarks(img, hand, self.mpHands.HAND_CONNECTIONS,
                                          self.mpDraw.DrawingSpec(color=(r, g, b), thickness=circle_size, circle_radius=circle_size),
                                          self.mpDraw.DrawingSpec(color=(r_, g_, b_), thickness=line_size, circle_radius=line_size),
                                          )

        return


def main():

    # Declare time variables
    pTime = 0
    cTime = 0
    # Show video capture with Open CV
    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
    # Set size of video window
    cap.set(3, wCam)
    cap.set(4, hCam)
    # Run main class which detects hands
    detector = handDetector()

    # Declare variables that will determine which poses are being triggers
    mudra = False

    aakash_pose_left = False
    aakash_pose_right = False
    prithvi_pose_left = False
    prithvi_pose_right = False
    surya_pose_left = False
    surya_pose_right = False
    varuna_pose_left = False
    varuna_pose_right = False
    vayu_pose_left = False
    vayu_pose_right = False

    aakash_full = False
    prithvi_full = False
    surya_full = False
    varuna_full = False
    vayu_full = False

    while True:
        # Create video object frame
        success, img = cap.read()

        # flip image
        img = cv2.flip(img, 1)
        img = detector.findHands(img)

        # Pass all hand landmark values from findPosition functions to arrays
        lmListLeft = detector.findPositionLeft(img)
        lmListRight = detector.findPositionRight(img)

        # Use getHand functino to determine which hand is appearing in video capture
        right = detector.getRightHand(img)
        left = detector.getLeftHand(img)

        # If mudra is not activated use below to render default connections and circles
        if not mudra:
            detector.render(img, randrange(255), randrange(255), randrange(255), randrange(255), randrange(255), randrange(255), 1, 1)

        if right:

            if len(lmListRight) != 0:

                # Set necessary landmarks for mudras as variables
                rX3, rY3 = lmListRight[3][1], lmListRight[3][2]
                rX4, rY4 = lmListRight[4][1], lmListRight[4][2]
                rX7, rY7 = lmListRight[7][1], lmListRight[7][2]
                rX8, rY8 = lmListRight[8][1], lmListRight[8][2]
                rX12, rY12 = lmListRight[12][1], lmListRight[12][2]
                rX14, rY14 = lmListRight[14][1], lmListRight[14][2]
                rX16, rY16 = lmListRight[16][1], lmListRight[16][2]
                rX20, rY20 = lmListRight[20][1], lmListRight[20][2]

                if -60<rX4-rX12<24 and -10<rY4-rY12<24:

                    # User is making Aakash pose with right hand
                    aakash_pose_right = True
                    prithvi_pose_right = False
                    surya_pose_right = False
                    varuna_pose_right = False
                    vayu_pose_right = False
                    print("aakash_pose_right is true")

                elif -10<rX4-rX16<5 and -20<rY4-rY16<5:

                    # User is making Prithvi pose with right hand
                    aakash_pose_right = False
                    prithvi_pose_right = True
                    surya_pose_right = False
                    varuna_pose_right = False
                    vayu_pose_right = False
                    print("prithvi_pose_right is true", rX4-rX16, " y:",rY4-rY16)

                elif rX4-rX16<(-5) and rY4-rY16<(-35) :

                    # User is making Surya pose with right hand
                    aakash_pose_right = False
                    prithvi_pose_right = False
                    surya_pose_right = True
                    varuna_pose_right = False
                    vayu_pose_right = False
                    print("surya_pose_right is true")

                elif (-40)<rX4-rX20<(-20) and (-10)<rY4-rY20<(10) :

                    # User is making Varuna pose with right hand
                    aakash_pose_right = False
                    prithvi_pose_right = False
                    surya_pose_right = False
                    varuna_pose_right = True
                    vayu_pose_right = False
                    print("varuna_pose_right is true", rX4 - rX20, " y:", rY4 - rY20)

                elif -25<rX3-rX8<15 and 20<rY3-rY8<60:

                    # User is making Vayu pose with right hand
                    aakash_pose_right = False
                    prithvi_pose_right = False
                    surya_pose_right = False
                    varuna_pose_right = False
                    vayu_pose_right = True
                    print("vayu_pose_right is true", rX3 - rX8, " y:", rY3 - rY8)

                # else:
                    # print("4-16-right x:", rX4 - rX16, " y:", rY4 - rY16)
                    # print("4-20-right x:", rX4 - rX20, " y:", rY4 - rY20)
                    # print("3-8-right x:", rX3 - rX8, " y:", rY3 - rY8)
            else:

                # If no poses are activated set all to false
                aakash_pose_right = False
                prithvi_pose_right = False
                surya_pose_right = False
                varuna_pose_right = False
                vayu_pose_right = False

        if left:

            if len(lmListLeft) != 0:

                # Set necessary landmarks for mudras as variables
                lX3, lY3 = lmListLeft[3][1], lmListLeft[3][2]
                lX4, lY4 = lmListLeft[4][1], lmListLeft[4][2]
                lX7, lY7 = lmListLeft[7][1], lmListLeft[7][2]
                lX8, lY8 = lmListLeft[8][1], lmListLeft[8][2]
                lX12, lY12 = lmListLeft[12][1], lmListLeft[12][2]
                lX14, lY14 = lmListLeft[14][1], lmListLeft[14][2]
                lX16, lY16 = lmListLeft[16][1], lmListLeft[16][2]
                lX20, lY20 = lmListLeft[20][1], lmListLeft[20][2]

                if -60<lX4-lX12<24 and -10<lY4-lY12<24:

                    # User is making Aakash pose with left hand
                    aakash_pose_left = True
                    prithvi_pose_left = False
                    surya_pose_left = False
                    varuna_pose_left = False
                    vayu_pose_left = False
                    print("aakash_pose_left is true")
                    print("8-16=x:", lX8-lX16, " y:",lY8-lY16)

                elif -10<lX4-lX16<5 and -20<lY4-lY16<5:

                    # User is making Prithvi pose with left hand
                    aakash_pose_left = False
                    prithvi_pose_left = True
                    surya_pose_left = False
                    varuna_pose_left = False
                    vayu_pose_left = False
                    print("prithvi_pose_left is true 8-16=x:", lX4-lX16, " y:",lY4-lY16)

                elif lX4-lX16<-5 and lY4-lY16<-20:

                    # User is making Surya pose with left hand
                    aakash_pose_left = False
                    prithvi_pose_left = False
                    surya_pose_left = True
                    varuna_pose_left = False
                    vayu_pose_left = False
                    print("surya_pose_left is true")

                elif (-10)<lX4-lX20<10 and (-40)<lY4-lY20<-(20):

                    # User is making Varuna pose with left hand
                    aakash_pose_left = False
                    prithvi_pose_left = False
                    surya_pose_left = False
                    varuna_pose_left = True
                    vayu_pose_left = False
                    print("varuna_pose_left is true 4-20=x:", lX4-lX20, " y:",lY4-lY20)

                elif 0<lX3-lX8<35 and 0<lY3-lY8<20:

                    # User is making Vayu pose with left hand
                    aakash_pose_left = False
                    prithvi_pose_left = False
                    surya_pose_left = False
                    varuna_pose_left = False
                    vayu_pose_left = True
                    print("vayu_pose_left is true", "3-8=x:", lX3-lX8, " y:",lY3-lY8)

                # else:
                    # print("4-12-left x:", lX4 - lX12, " y:", lY4 - lY12)
                    # print("4-16-left x:", lX4 - lX16, " y:", lY4 - lY16)
                    # print("4-20-left x:", lX4 - lX20, " y:", lY4 - lY20)
                    # print("3-7-left x:", lX3 - lX8, " y:", lY3 - lY8)
            else:

                # If no poses are activated set all to false
                aakash_pose_left = False
                prithvi_pose_left = False
                surya_pose_left = False
                varuna_pose_left = False
                vayu_pose_left = False

        if aakash_pose_left is True and aakash_pose_right is True:

            # Aakash pose is being made with both hands
            # Trigger Aakash effect

            mudra = True

            aakash_full = True
            prithvi_full = False
            surya_full = False
            varuna_full = False
            vayu_full = False

            space_color_0 = randrange(85)
            space_color_1 = randrange(85)

            detector.render(img, space_color_0, space_color_0, space_color_0, space_color_1, space_color_1,
                            space_color_1, randrange(1, 10, 1), randrange(1, 10, 1))

            cv2.rectangle(img, (10, 10), (300, 80), (space_color_0, space_color_0, space_color_0), -1)
            cv2.putText(img, "AAKASH", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 2.5,
                        (255, 255, 255), 3)

            cv2.rectangle(img, (0, 620), (1280, 720), (space_color_0, space_color_0, space_color_0), -1)
            cv2.putText(img, "Increasing the element of space, or cosmos, within the body helps", (50, 660),
                        cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (255, 255, 255), 3)
            cv2.putText(img, "the practitioner to unite with a specific deity or a broader cosmic power", (50, 700),
                        cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (255, 255, 255), 3)

        elif prithvi_pose_left is True and prithvi_pose_right is True:

            # Prithvi pose is being made with both hands
            # Trigger Prithvi effect

            mudra = True

            aakash_full = False
            prithvi_full = True
            surya_full = False
            varuna_full = False
            vayu_full = False

            earth_color_0 = randrange(87, 117, 1)
            earth_color_1 = randrange(126, 156, 1)
            earth_color_2 = randrange(45, 75, 1)

            earth_color_0_ = randrange(49, 79, 1)
            earth_color_1_ = randrange(64, 94, 1)
            earth_color_2_ = randrange(21, 51, 1)

            detector.render(img, earth_color_0_, earth_color_1_, earth_color_2_, earth_color_0_, earth_color_1,
                            earth_color_2_, randrange(1, 10, 1), randrange(1, 10, 1))

            cv2.rectangle(img, (10, 10), (300, 80), (60, 141, 102), -1)
            cv2.putText(img, "PRITHVI", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 2.5,
                        (255, 255, 255), 3)

            cv2.rectangle(img, (0, 620), (1280, 720), (60, 141, 102), -1)
            cv2.putText(img, "Increasing  the Earth element within an individual as a means of healing.", (50, 660),
                        cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (255, 255, 255), 3)
            cv2.putText(img, "When the earth element is in balance, we are both mentally and physically strong and stable.", (50, 700),
                        cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (255, 255, 255), 3)

        elif surya_pose_left is True and surya_pose_right is True:

            # Surya pose is being made with both hands
            # Trigger Surya effect

            mudra = True

            prithvi_full = False
            aakash_full = False
            surya_full = True
            varuna_full = False
            vayu_full = False

            fire_color_0 = randrange(0, 15, 1)
            fire_color_1 = randrange(19, 49, 1)
            fire_color_2 = randrange(167, 197, 1)

            fire_color_0_ = randrange(0, 15, 1)
            fire_color_1_ = randrange(85, 112, 1)
            fire_color_2_ = randrange(237, 287, 1)

            detector.render(img, fire_color_0, fire_color_1, fire_color_2, fire_color_0_, fire_color_1_,
                            fire_color_2_, randrange(1, 10, 1), randrange(1, 10, 1))

            cv2.rectangle(img, (10, 10), (300, 80), (fire_color_0, fire_color_1, fire_color_2), -1)
            cv2.putText(img, "SURYA", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 2.5,
                        (255, 255, 255), 3)

            cv2.rectangle(img, (0, 620), (1280, 720), (fire_color_0, fire_color_1, fire_color_2), -1)
            cv2.putText(img, "Increasing increases the fire element and eliminates the earth element in the body.", (50, 660),
                        cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (255, 255, 255), 3)
            cv2.putText(img, "A Sanskrit term, surya means 'sun' and mudra means 'gesture'." , (50, 700),
                        cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (255, 255, 255), 3)

        elif varuna_pose_left is True and varuna_pose_right is True or varuna_pose_right is True and varuna_pose_left is True:

            # Varuna pose is being made with both hands
            # Trigger Varuna effect

            mudra = True

            prithvi_full = False
            aakash_full = False
            surya_full = False
            varuna_full = True
            vayu_full = False

            water_color_0 = randrange(209, 229, 1)
            water_color_1 = randrange(201, 221, 1)
            water_color_2 = randrange(146, 166, 1)

            water_color_0_ = randrange(240, 255, 1)
            water_color_1_ = randrange(240, 255, 1)
            water_color_2_ = randrange(0, 20, 1)

            detector.render(img, water_color_0, water_color_1, water_color_2, water_color_0_, water_color_1_,
                            water_color_2_, randrange(1, 10, 1), randrange(1, 10, 1))

            cv2.rectangle(img, (10, 10), (300, 80), (water_color_0, water_color_1, water_color_2), -1)
            cv2.putText(img, "VARUNA", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 2.5,
                        (255, 255, 255), 3)

            cv2.rectangle(img, (0, 620), (1280, 720), (water_color_0, water_color_1, water_color_2), -1)
            cv2.putText(img, "Designed to balance the water element in the body.", (50, 660),
                        cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (255, 255, 255), 3)
            cv2.putText(img, "Named from the Sanskrit Varun, which is the name of the Hindu god of water," , (50, 700),
                        cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (255, 255, 255), 3)

        elif vayu_pose_left is True and vayu_pose_right is True or vayu_pose_right is True and vayu_pose_left is True:

            # Vayu pose is being made with both hands
            # Trigger Vayu effect

            mudra = True

            prithvi_full = False
            aakash_full = False
            surya_full = False
            varuna_full = False
            vayu_full = True

            air_color_0 = randrange(170, 255, 1)
            air_color_1 = randrange(170, 255, 1)

            detector.render(img, air_color_0, air_color_0, air_color_0, air_color_1, air_color_1,
                            air_color_1, randrange(1, 10, 1), randrange(1, 10, 1))

            cv2.rectangle(img, (10, 10), (250, 80), (air_color_1, air_color_1, air_color_1), -1)
            cv2.putText(img, "VAYU", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 2.5,
                        (0, 0, 0), 3)

            cv2.rectangle(img, (0, 620), (1280, 720), (air_color_1, air_color_1, air_color_1), -1)
            cv2.putText(img, "The name comes from the Sanskrit, vayu, meaning 'air'.", (50, 660),
                        cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (0, 0, 0), 3)
            cv2.putText(img, "the air element is suppressed by the fire of the thumb in this mudra" , (50, 700),
                        cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (0, 0, 0), 3)
        else:
            mudra = False

        # Determine current time
        cTime = time.time()
        # Determine frames per second
        fps = 1 / (cTime - pTime)
        # Previous time becomes curent time
        pTime = cTime

        # Uncomment if you want to view frames per second
        # cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 3)

        # Run webcam
        cv2.imshow("Image", img)
        # cv2.waitKey(1)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

if __name__ == "__main__":
    main()

cap.release()
cv2.destroyAllWindows()
    
